In [1]:
import pandas as pd
from moviepy.editor import *
from PIL import Image
from PIL import ImageDraw
from numpy import asarray

In [ ]:
# Change FFMPEG binary path to allow for GPU acceleration.
from moviepy.config import change_settings
change_settings({"FFMPEG_BINARY":"ffmpeg"})

In [2]:
# this can be empty if the video file and its videopipe output are at the same
# location as the code
path = ''
v_name = 'HIGH_LIGHTS_I_SNOWMAGAZINE_I_SANDER_26'
task = '_shot_boundaries_datamodel'
w, h = 1920, 1080
RESIZE_DIM = 640

In [3]:
## read shots detection json

shots = pd.read_json(path + v_name + '/' + v_name + task + '.json', lines = True)

shots_detected = [f for f in shots.data[0]]

In [4]:
## read video file with moviepy

clip = VideoFileClip(v_name + '.mp4')

fps = clip.fps
frame_duration = 1 / fps

# All frames in json file
frames = [f['dimension_idx'] for f in shots_detected]

Create textclips for each entry in JSON.

In [6]:
txtclips = []

# Set text to display, appended by number of shot.
txt = 'Shot'

# Set duration of text clip
txtclip_dur = 1

count = 1

for f in frames:
    txtclip = (TextClip(txt + " " + str(count), color='white',
            font="Century-Schoolbook-Roman", fontsize=70, kerning=-2,
            interline=-1, bg_color='black', size = (w, h))
            .set_duration(txtclip_dur)
            .set_position(('center')))
    txtclips.append(txtclip)
    count += 1


Create list of subclips. First, a textclip is added showing the number of the shot. Next, a subclip is added containing the entire shot. Doing this for all shots results in a list of correctly ordered text- and subclips which can then be concatenated and written to an mp4 file

In [7]:
subclips = []

timestamps = [f * frame_duration for f in frames]

for i in range(len(timestamps) - 1):
    subclip = clip.subclip(timestamps[i], timestamps[i + 1])
    subclips.append(txtclips[i])
    subclips.append(subclip)

# Add the last text clip and the last subclip
subclips.append(txtclips[-1])
subclip = clip.subclip(timestamps[-1], clip.duration)
subclips.append(subclip)

final = concatenate_videoclips(subclips)

In [8]:
final.write_videofile("shot.mp4", codec='h264_nvenc', fps=25)

Moviepy - Building video shot.mp4.
MoviePy - Writing audio in shotTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video shot.mp4



Moviepy - Done !
Moviepy - video ready shot.mp4
